# **Global Constants**

In [0]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/gdrive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Teaching/2019-20-BDC/datasets"
DATASET_URL = "https://github.com/gtolomei/big-data-computing/raw/master/datasets/king-county-house-sales.csv.bz2"
GDRIVE_DATASET_FILE = GDRIVE_DATA_DIR + "/" + DATASET_URL.split("/")[-1]

RANDOM_SEED = 42 # for reproducibility

# **Spark + Google Colab Setup**

## **1.** Install PySpark and related dependencies

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

## **2.** Import useful Python packages

In [0]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## **3.** Create Spark context

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## **4.** Create <code>ngrok</code> tunnel to check the Spark UI

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## **5.** Link Colab to our Google Drive

In [0]:
# Point Colaboratory to our Google Drive

from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

## **6.** Check everything is ok

In [0]:
spark

In [0]:
sc._conf.getAll()

# **The Prediction Task**

In this notebook, we will be using a dataset from [Kaggle](https://www.kaggle.com/burhanykiyakoglu/predicting-house-prices/data) containing house sale records from [King County, WA, USA](https://en.wikipedia.org/wiki/King_County,_Washington) to predict house prices.

Intuitively, each sold house is described by a set of properties (i.e., _features_) and the price it has been eventually sold. Our goal is to use those features (or, at least, a subset of them) to accurately predict the price which an _unseen_ house should be sold.

In the following, we implement each step of the pipeline which we have discussed in our classes.

# **1. Data Collection**

This is the first step we need to accomplish before going any further. The dataset will be downloaded directly to our Google Drive, as usual.

### **Download dataset file from URL directly to our Google Drive**

In [0]:
def get_data(dataset_url, dest, chunk_size=1024):
  response = requests.get(dataset_url, stream=True)
  if response.status_code == 200:
    with open(dest, "wb") as file:
      for block in response.iter_content(chunk_size=chunk_size): 
        if block: 
          file.write(block)

In [0]:
print("Retrieving dataset from URL: {} ...".format(DATASET_URL))
get_data(DATASET_URL, GDRIVE_DATASET_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_FILE))

### **Read dataset file into a Spark Dataframe**

In [0]:
house_df = spark.read.load(GDRIVE_DATASET_FILE, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

### **Check the shape of the loaded dataset, i.e., number of rows and columns**

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(house_df.count(), len(house_df.columns)))

### **Print out the schema of the loaded dataset**

In [0]:
house_df.printSchema()

### **Cast `price` type to `double` to avoid compatibility issues**

In [0]:
house_df = house_df.withColumn("price", house_df["price"].cast("double"))

### **Dataset Shape and Schema**

The dataset contains **21,613** historical records of house sales; each record, contains the following set of **21** columns:
- `id`: unique ID for each house sold (_numerical_, _discrete_);
- `date`: Date when the house was sold (_datetime_);
- **`price`**: The price a home has been sold (_numerical_, _continuous_); **[This is the _target_ variable we want to predict]**
- `bedrooms`: Number of bedrooms (_numerical_, _discrete_);
- `bathrooms`: Number of bathrooms (_numerical_, _continuous_) [.5 accounts for a room with a toilet but no shower];
- `sqft_living`: Square footage of the house's interior living space (_numerical_, _continuous_);
- `sqft_lot`: Square footage of the house's land space (_numerical_, _continuous_);
- `floors`: Number of floors (_numerical_, _discrete_);
- `waterfront`:  A dummy (i.e., binary) variable for whether the house is overlooking the waterfront or not (_categorical_, _nominal_);
- `view`: An index from 0 to 4 of how good the view of the property was (_categorical_, _ordinal_);
- `condition`: An index from 1 to 5 on the condition of the house (_categorical_, _ordinal_);
- `grade`: An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design (_categorical_, _ordinal_);
- `sqft_above`: The square footage of the interior housing space that is above ground level (_numerical_, _continuous_);
- `sqft_basement`: The square footage of the interior housing space that is below ground level (_numerical_, _continuous_);
- `yr_built`: The year the house was initially built (_numerical_, _discrete_);
- `yr_renovated`: The year of the house's last renovation (_numerical_, _discrete_);
- `zipcode`: The zipcode area where the house is located (_categorical_, _nominal_);
- `lat`: Latitude (_numerical_, _continuous_);
- `long`: Longitude (_numerical_, _continuous_);
- `sqft_living15`: The square footage of interior housing living space for the nearest **15** neighbors (_numerical_, _continuous_);
- `sqft_lot15`: The square footage of the land lots of the nearest **15** neighbors (_numerical_, _continuous_).



In [0]:
# Let's define some constants which we will use throughout this notebook
INDEX_FEATURE = "id"
NUMERICAL_FEATURES = ["bedrooms", 
                      "bathrooms",
                      "floors",
                      "lat",
                      "long",
                      "sqft_above", 
                      "sqft_basement",
                      "sqft_living", 
                      "sqft_living15", 
                      "sqft_lot", 
                      "sqft_lot15",
                      "yr_built",
                      "yr_renovated"
                      ]
CATEGORICAL_FEATURES = ["condition", "grade", "view", "waterfront", "zipcode"]
TARGET_VARIABLE = "price"

In [0]:
print("1 Index feature = `{:s}`".format(INDEX_FEATURE))
print("{:d} Numerical features = [{:s}]".format(len(NUMERICAL_FEATURES), ", ".join(["`{:s}`".format(nf) for nf in NUMERICAL_FEATURES])))
print("{:d} Categorical features = [{:s}]".format(len(CATEGORICAL_FEATURES), ", ".join(["`{:s}`".format(nf) for nf in CATEGORICAL_FEATURES])))
print("1 Target variable = `{:s}`".format(TARGET_VARIABLE))

### **Display the first 5 rows of the dataset**

In [0]:
house_df.show(5)

### **Check for any missing values**

In [0]:
for c in house_df.columns:
  print("N. of missing values of column `{:s}` = {:d}".format(c, house_df.where(col(c).isNull()).count()))

# **2. Data Exploration**

Before starting with the application of any (linear regression) modeling technique, a good practice is to first take a look at a few statistics computed from the data. In addition, drawing specific plots may help us spot interesting facts (e.g., the presence of outliers), which in turn could indicate us what steps to do next (e.g., outliers removal/winsorization). 

### **Summary of Descriptive Statistics**

In [0]:
house_df.describe().show()

In [0]:
# To access plotting libraries, we need to first transform our PySpark DataFrame into a Pandas DataFrame
house_pdf = house_df.toPandas() 

In [0]:
# Set some default plotting configuration using seaborn properties
sns.set_style("darkgrid")
sns.set_context("notebook", rc={"lines.linewidth": 2, 
                                "xtick.labelsize":14, 
                                "ytick.labelsize":14,
                                "axes.labelsize": 18
                                })

### **Distribution Plots**

This set of plots will show the distribution of values of a subset of **19** columns of interest. The following feature will **not** be included:
- `id`
- `date`

In [0]:
# Select only the columns which will be included in the distribution plots
PLOT_COLUMNS = sorted(NUMERICAL_FEATURES + CATEGORICAL_FEATURES + [TARGET_VARIABLE])
print("Plotting {:d} columns from the dataset:\n[{:s}]".format(len(PLOT_COLUMNS), 
                                                              ", ".join(["`{:s}`".format(pc) for pc in PLOT_COLUMNS])
                                                              ))

In [0]:
# Plot the distribution of values of each column of interest
n_rows = 5
n_cols = 4

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20,20))
fig.delaxes(axes[4][3]) # the last ax will be empty as there are 19 columns for 20 slots, so just delete it!

for i,f in enumerate(PLOT_COLUMNS):
  _ = sns.distplot(house_pdf[f],
                   kde_kws={"color": "#ca0020", "lw": 1}, 
                   hist_kws={"histtype": "bar", "edgecolor": "k", "linewidth": 1,"alpha": 0.8, "color": "#92c5de"},
                   ax=axes[i//n_cols, i%n_cols]
                   )

fig.tight_layout(pad=1.5)

### **Relationship between _numerical continuous_ features and the _target variable_ (`price`)**

In [0]:
NUMERICAL_CONTINUOUS_FEATURES = sorted(list(set(NUMERICAL_FEATURES) - set(["bedrooms", "floors", "yr_renovated"])))
print("Regression plots of {:d} numerical continuous features:\n[{:s}]".format(len(NUMERICAL_CONTINUOUS_FEATURES),
                                                                              ", ".join(["`{:s}`".format(ncf) for ncf in NUMERICAL_CONTINUOUS_FEATURES])
                                                                              ))

In [0]:
# Plot the relationship between each continuous feature (i.e., independent variable) with the target (i.e., dependent) variable
n_rows = 2
n_cols = 5

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20,14))

for i,f in enumerate(NUMERICAL_CONTINUOUS_FEATURES):
  _ = sns.regplot(data=house_pdf, 
                  x=f, 
                  y="price",
                  color="#00b159",
                  ax=axes[i//n_cols, i%n_cols])

fig.tight_layout(pad=1.5)

#### **Observations**

It seems there is a moderate linear relationship between `sqft_living` and `price`. Still, the former feature _alone_ might not be enough to correctly explain observed values of `price`.

### **Relationship between _numerical discrete_/_categorical_ features and the _target variable_ (`price`)**

In [0]:
DISCRETE_FEATURES = sorted(list(set(CATEGORICAL_FEATURES + ["bedrooms", "bathrooms", "floors"]) - set(["zipcode"])))
print("Box plots of {:d} discrete features:\n[{:s}]".format(len(DISCRETE_FEATURES),
                                                                              ", ".join(["`{:s}`".format(df) for df in DISCRETE_FEATURES])
                                                                              ))

In [0]:
# Plot the relationship between each discrete feature (i.e., independent variable) with the target (i.e., dependent) variable
n_rows = 3
n_cols = 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16,16))

for i,f in enumerate(DISCRETE_FEATURES[1:]):
  _ = sns.boxplot(data=house_pdf,
                  x=f, 
                  y="price", 
                  ax=axes[i//n_cols, i%n_cols])
  
fig.tight_layout(pad=1.5)

# Plot `bathrooms` on a dedicated chart
fig, ax = plt.subplots(1, 1, figsize=(16,8))
_ = sns.boxplot(data=house_pdf, x="bathrooms", y="price")

### **Correlation between features**

We know that having too many features in a model is not always a good thing because it might cause _overfitting_ and therefore bad generalization performance to unseen examples. Thus, if a feature does not improve our model a lot, removing it may indeed be a better choice.

Things get even worst if there exists very high **correlation** between a subset of features, as keeping all of them most of the time will again cause overfitting. For instance, if we figure out that `sqt_above` is highly correlated with `sqt_living` there is no point for us to keep them both, as one can be easily derived from the other and together they do not add any valuable predictive power to our model.

The presence of high correlation (whether it be positive or negative) can be checked by computing and visualizing the **correlation matrix**. However, this does not mean that we must _always_ remove one of the highly correlated features. In some cases, even if two features are highly correlated they still may capture different aspects of the domain objects.

### **Pearson Correlation Coefficient and Correlation Matrix**

Remember that given 2 random variables $X$ and $Y$, their _Pearson Correlation Coefficient_ $\rho(X, Y)$ can be computed as follows:

$$
\rho(X, Y) = Cov(X_{\text{std}}, Y_{\text{std}}) = E[X_{\text{std}} - E[X_{\text{std}}]]*E[Y_{\text{std}} - E[Y_{\text{std}}] ] = 
$$
$$
=E[X_{\text{std}}]*E[Y_{\text{std}}] = E\Bigg[\frac{(X - \mu_X)}{\sigma_X}\Bigg] * E\Bigg[\frac{(Y - \mu_Y)}{\sigma_Y}\Bigg] =
$$
$$
=\frac{E[X - \mu_X]*E[Y - \mu_Y]}{\sigma_X \sigma_Y} = \frac{Cov(X,Y)}{\sigma_X \sigma_Y}
$$

Eventually, the **correlation matrix** $M$ is the triangular square matrix such that $M[i][j] = \rho(X_i, X_j)$

In [0]:
# Select the features we want to use to compute the correlation matrix (i.e., everything except `id` and `date`)
features = [TARGET_VARIABLE] + sorted(NUMERICAL_FEATURES + CATEGORICAL_FEATURES)

mask = np.zeros_like(house_pdf[features].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

with sns.axes_style("white"): # Temporarily set the background to white
  fig, ax = plt.subplots(figsize=(16, 12))
  plt.title('Pearson Correlation Matrix', fontsize=24)

  cmap = sns.diverging_palette(220, 10, as_cmap=True)

  _ = sns.heatmap(house_pdf[features].corr(), 
              linewidths=0.25, 
              vmax=0.7, 
              square=True,
              ax=ax, 
              cmap=cmap, 
              linecolor='w', 
              annot=True, 
              annot_kws={"size":8}, 
              mask=mask, 
              cbar_kws={"shrink": .9});

#### **Observations**

As expected, `sqft_living` and `sqft_above` are high-positively correlated with each other ($\rho = 0.88$). Similarly, also `bathrooms`/`bedrooms` and `sqft_living` are high-positively correlated with each other ($\rho=0.75$ and $0.70$, respectively). Again, this seems totally legitimate, as the number of bathrooms and bedrooms depends on the square footage of a house.

Later on, we will see how to deal with this multicollinearity between features, and what will be the effect on the quality of the learned model when those features are retained or removed.

# **3. The Learning Pipeline**

### **Important Remark on Dataset Transformations**

Any dataset transformation we want to perform must be **derived** from the _training set_ portion of the original dataset, and then applied to (_validation_ and) _test set_.

Just to make an example, if we need to standardize our feature values we will also need to compute the **mean** and the **standard deviation** of each feature. Now, those statistics **cannot** be computed by looking at the whole dataset. In other words, we first need to split our dataset into at least two portions: _training_ and _test set_. Then, we will compute the mean and standard deviation of all the feature values from the observations contained in the training set, and standardize those values accordingly. Of course, we will need also to standardize features appearing in the test set, and in order to do that you will use the means and standard deviations **as previoulsly computed from the training set**.

This is just following the general principle: anything we learn must be learned from the model's training data.

### **Dataset Splitting: Training vs. Test Set**

Before moving along with any preprocessing involving data transformations, we will split our dataset into **2** portions:
- _training set_ (e.g., accounting for 90% of the total number of instances);
- _test set_ (e.g., accounting for the remaining 10% of instances)

In [0]:
# Randomly split our original dataset `house_df` into 90÷10 for training and test, respectively
train_df, test_df = house_df.randomSplit([0.9, 0.1], seed=RANDOM_SEED)

### **Working on the Training Set only**

From now on, we will be working on the training set portion only. The test set will come back into play when we evaluate our learned model.

## **The Quickest Linear Regression Model**

Initially, we try to learn a simple (i.e., _univariate_) linear regression model. To this end, we select only one feature from the original set, which we believe it is mostly related to our target variable `price`.
As an example, suppose we choose `sqft_living`.

Our linear regression model would thus look like the following:

$$
\underbrace{h_{\theta}(x)}_{\texttt{price}} = \theta_0 + \theta_1 \cdot \underbrace{x}_{\texttt{sqft_living}}
$$

### **Extract the single predictor variable (`sqft_living`) and the target variable (`price`)**

In [0]:
# Select `sqft_living` feature only together with the target variable `price` from the original `train_df`
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["sqft_living"], 
                            outputCol="features")

simple_train_df = assembler.transform(train_df)

In [0]:
# Select only the two columns which we are interested in, i.e., `features` and `price`
simple_train_df = simple_train_df.select(["features", "price"])
simple_train_df.show(5, truncate=False)

### **The General Optimization Problem behind Linear Regression**
Given an $m$-by-$n+1$ **feature matrix** $\mathbf{X}$, an $m$-by-1 **target vector** $\mathbf{y}$, and an $n+1$-by-1 **parameter vector** $\boldsymbol{\theta}$ associated with a hypothesis function defined as $h_{\boldsymbol{\theta}}(\mathbf{x}): \theta_0 + \theta_1 x_1 + \ldots + \theta_n x_n = \mathbf{x}^T\cdot \boldsymbol{\theta}$

In its simplest form (a.k.a. **Ordinary Least Squares** or **OLS**), the linear regression problem aims to find the **parameter vector** $\boldsymbol{\theta}^{*}$ which minimizes the **Mean Squared Error**.

$$
\boldsymbol{\theta}^* = \text{argmin}_{\boldsymbol{\theta}\in \mathbb{R}^n} \underbrace{\frac{1}{m} ||\mathbf{X}\cdot \boldsymbol{\theta} -\mathbf{y}||^2}_{\text{Mean Squared Error}}
$$

OLS is a special case of a more general optimization framework, which is known as **Elastic Net** and is defined as follows:

$$
\boldsymbol{\theta}^* = \text{argmin}_{\boldsymbol{\theta}\in \mathbb{R}^n} \frac{1}{m} ||\mathbf{X}\cdot \boldsymbol{\theta} -\mathbf{y}||^2 + \lambda\Big(\alpha |\boldsymbol{\theta}| + (1-\alpha)||\boldsymbol{\theta}||^2\Big)
$$
where:
- $\lambda \geq 0$ is the **regularization parameter**;
- $\alpha \in [0,1]$ is the **tradeoff parameter** for the regularization penalties;
- $|\boldsymbol{\theta}|$ is the L1-norm of the **parameter vector**;
- $||\boldsymbol{\theta}||^2$ is the squared L2-norm (i.e., squared Euclidean norm) of the **parameter vector**.

Depending on the values that are plugged in the general optimization framework above, we have the following cases:

- $\lambda = 0 \Rightarrow $ **OLS** (no regularization at all);
- $\lambda > 0$ and $\alpha = 1 \Rightarrow$ **Least Absolute Shrinkage and Selection Operator** or **LASSO** (L1-regularization only);
- $\lambda > 0$ and $\alpha = 0 \Rightarrow$ **Ridge** (L2-regularization only);
- Any other case where $\lambda > 0$ and $0 < \alpha < 1$ balances between L1- and L2-regularization).


### **Learn the model using the `LinearRegression` object**

We use the `LinearRegression` object provided within the `pyspark.ml.regression` package. Any further information can be found in the [API documentation](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression).

Among the parameters we can specify, let's have a look at the most important ones:
- `regParam` is the regularization parameter (or $\lambda$);
- `elasticNetParam` is the tradeoff parameter for regularization penalties (or $\alpha$);
  - `regParam = 0` and `elasticNetParam = 0` means there is no regularization (i.e., OLS);
  - `regParam > 0` and `elasticNetParam = 0` means there is only L2-regularization (Ridge); 
  - `regParam > 0` and `elasticNetParam = 1` means there is only L1-regularization (LASSO);
  - `regParam > 0` and `0 < elasticNetParam < 1` means there is both L1- and L2-regularization (Elastic Net);
- `solver` can take one of the following values: `l-bfgs`, `normal` and `auto`. 
  - `l-bfgs` stands for **Limited-memory BFGS** which is a limited-memory quasi-Newton optimization _iterative_ method; 
  - `normal` refers to **Normal Equation** _analytical_ method, which computes the solution to the optimization problem _directly_ using the **pseudo-inverse** of the feature matrix and multiplying it by the target vector;
  - `auto` leaves PySpark to select the solver algorithm automatically.

### **Training a simple univariate OLS linear regressor (with no regularization)**

In [0]:
from pyspark.ml.regression import LinearRegression

# First of all, let's just setup a very basic OLS linear regressor (i.e., univariate with no regularizatio term)
lr = LinearRegression(featuresCol="features", labelCol="price")
# Train the actual model on our training set `simple_train_df`
lr_model = lr.fit(simple_train_df)

#### **Intercept ($\theta_0$) and Coefficient ($\theta_1$)**

In [0]:
print("Intercept: " + str(lr_model.intercept))
print("Coefficient: " + str(lr_model.coefficients))

#### **Measuring performance on the Training Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
training_result = lr_model.summary
print("***** Training Set *****")
print("RMSE: {:.3f}".format(training_result.rootMeanSquaredError))
print("R2: {:.3f}".format(training_result.r2))
print("Adjusted R2: {:.3f}".format(training_result.r2adj))
print("***** Training Set *****")

### **Testing the simple univariate OLS linear regressor learned above on the Test Set**

In [0]:
# Select `sqft_living` feature only together with the target variable `price` from the original `train_df`
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["sqft_living"], 
                            outputCol="features")

simple_test_df = assembler.transform(test_df)

In [0]:
# `lr_model` is a Transformer which can be used to "transform" our test set
lr_predictions = lr_model.transform(simple_test_df) 

In [0]:
# `lr_predictions` is a dataframe containing (among other things) the predictions made by `lr_model` on the test set
lr_predictions.select("features", "prediction", "price").show(5)

#### **Measuring performance on the Test Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
test_result = lr_model.evaluate(simple_test_df)
print("***** Test Set *****")
print("RMSE: {:.3f}".format(test_result.rootMeanSquaredError))
print("R2: {:.3f}".format(test_result.r2))
print("Adjusted R2: {:.3f}".format(test_result.r2adj))
print("***** Test Set *****")

#### **Observations**

It turns out that the very simple univariate linear regressor does not work well. This may be due to the fact that a single explanatory variable (feature) alone like `sqft_living` could be not enough to nicely predict our target variable `price`.

Also, the fact that both $R^2$ and $R^2_{\text{adj}}$ are slightly better on the test set than on the training set may be simply due to a "lucky" random split of the initial dataset.

## **Summary of the main findings from Data Exploration step**

Let's now go back to the result of our exploratory data analysis above, where we were able to observe the following:
- Both continuous and categorical features;
- Different scales of feature values;
- Several outliers on many features (see the boxplots above).

### **Transform Categorical features into Numerical using One-Hot Encoding**

Let's address the first issue indicated above.

To transform _categorical_ features into _numerical_ ones we proceed as follows.
We setup a pipeline which is composed of the following steps:
- [`StringIndexer`](https://spark.apache.org/docs/latest/ml-features#stringindexer): encodes a string column of labels to a column of label indices. The indices are in `[0, numLabels)`, and 4 ordering options are supported (default `frequencyDesc`, which assigns the most frequent label the index `0`, and so on and so forth).
- [`OneHotEncoderEstimator`](https://spark.apache.org/docs/latest/ml-features#onehotencoderestimator): maps a categorical feature, represented as a label index, to a binary vector with at most a single one-value indicating the presence of a specific feature value from among the set of all feature values. An important parameter is `handleInvalid`, which indicates how to deal with previously unseen labels. By default this raises an error but it can be set to as `keep` to assign previously unseen labels a fallback value.
- [`VectorAssembler`](https://spark.apache.org/docs/latest/ml-features#vectorassembler): is a transformer that combines a given list of columns into a single vector column.

In [0]:
# This function is responsible to implement the pipeline above for transforming categorical features into numerical ones
def to_numerical(df, numerical_features, categorical_features, target_variable):

    """
    Args:
        - df: the input dataframe
        - numerical_features: the list of column names in `df` corresponding to numerical features
        - categorical_features: the list of column names in `df` corresponding to categorical features
        - target_variable: the column name in `df` corresponding to the target variable

    Return:
        - transformer: the pipeline of transformation fit to `df` (for future usage)
        - df_transformed: the dataframe transformed according to the pipeline
    """
    
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

    # 1. Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 2. Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")
    
    # 3. Assemble all the features into a single vector
    assembler = VectorAssembler(inputCols=encoder.getOutputCols() + numerical_features, outputCol="features")

    # 4. Setup the pipeline with the stages above
    pipeline = Pipeline(stages=indexers + [encoder] + [assembler])

    # 5. Transform the input dataframe accordingly
    transformer = pipeline.fit(df)
    df_transformed = transformer.transform(df)

    # 6. Optionally, change the name of the target variable to `label` (if that is different), as PySpark implicitly assumes this is the name of the column to predict
    if target_variable != "label":
        df_transformed = df_transformed.withColumnRenamed(target_variable, "label")

    # 7. Eventually, return both the transformed dataframe and the transformer object for future transformations
    return transformer, df_transformed 

In [0]:
# Transform the training set and get back both the transformer and the new dataset
oh_transformer, oh_train_df = to_numerical(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

In [0]:
# Show the result of numerical transformation
oh_train_df.show(5)

In [0]:
# Select `features` and `label` (i.e., formerly `price`) target variable only
train = oh_train_df.select(["features", "label"])

In [0]:
train.show(5, truncate=False)

### **Training an OLS Multivariate Linear Regressor (with no regularization) using all the features**

In [0]:
# `lr` is still the LinearRegressor object we set up above (the only difference here is that now `features` refers to the whole feature vector)
lr = LinearRegression(featuresCol="features", labelCol="label") # Analogous to lr = LinearRegression()
# Train the actual model on our training set `train`
lr_model = lr.fit(train)

### **Intercept ($\theta_0$) and Coefficients ($\theta_1, \ldots, \theta_n$)**

In [0]:
print("Intercept: " + str(lr_model.intercept))
print("Coefficients: " + str(lr_model.coefficients))

#### **Measuring performance on the Training Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
training_result = lr_model.summary
print("***** Training Set *****")
print("RMSE: {:.3f}".format(training_result.rootMeanSquaredError))
print("R2: {:.3f}".format(training_result.r2))
print("Adjusted R2: {:.3f}".format(training_result.r2adj))
print("***** Training Set *****")

### **Use the One-Hot encoding pipeline to transform the Test Set**

In [0]:
# Here, we use the same transformer as the one returned by the `to_numerical` function above yet applied to the test set
oh_test_df = oh_transformer.transform(test_df)

In [0]:
oh_test_df.show(5)

In [0]:
# Rename the target variable `price` to `label`
oh_test_df = oh_test_df.withColumnRenamed(TARGET_VARIABLE, "label")
oh_test_df.show(5)

In [0]:
# Select `features` and `label` only
test = oh_test_df.select(["features", "label"])
test.show(5)

### **Compute predictions on the Test Set according to the model learned on the Training Set**

In [0]:
# `lr_model` is a Transformer which can be used to "transform" our test set
lr_predictions = lr_model.transform(test)

In [0]:
# `lr_predictions` is a dataframe containing (among other things) the predictions made by `lr_model` on the test set
lr_predictions.select("features", "prediction", "label").show(5)

#### **Measuring performance on the Test Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
test_result = lr_model.evaluate(test)
print("***** Test Set *****")
print("RMSE: {:.3f}".format(test_result.rootMeanSquaredError))
print("R2: {:.3f}".format(test_result.r2))
print("Adjusted R2: {:.3f}".format(test_result.r2adj))
print("***** Test Set *****")

## **Summary of the main findings from Data Exploration step**

From our exploratory data analysis above, we observed:
- ~Both continuous and categorical features~ (**DONE**);
- Different scales of feature values;
- Several outliers on many features (see the boxplots above).

### **Standardize Features**

Let's now consider the second bullet of the list above: feature scales. We can use a feature scaling transformer to standardize all our features to 0-mean and 1-unit of standard deviation. 

In this way, the coefficient $\theta_0$ (i.e., the _intercept_) will have a nice interpretation, as it can be thought of as the prediction the model would make when _all_ the input features take on their respective mean values (i.e., 0).

In [0]:
# Let's define a function to standardize all the features
def standardize_features(df, input_col="features", with_std=True, with_mean=True):

    from pyspark.ml.feature import StandardScaler

    # 1. Create the StandardScaler
    scaler = StandardScaler(inputCol=input_col, outputCol="std_"+input_col, withStd=with_std, withMean=with_mean)

    # 2. Compute summary statistics by fitting the StandardScaler
    scaler_model = scaler.fit(df)

    # 3. Normalize each feature to have 0-mean and unit standard deviation
    scaled_data = scaler_model.transform(df)

    # 4. Eventually, return both the scaler_model (for future transformations) and the scaled data
    return scaler_model, scaled_data

In [0]:
# Call the function above
scaler_model, scaled_train = standardize_features(train)

In [0]:
scaled_train.select(["features", "std_features", "label"]).show(5, truncate=False)

In [0]:
# First of all, let's just setup an OLS linear regressor with no regularization term
lr = LinearRegression(featuresCol="std_features", labelCol="label")
# Train the actual model on our training set `scaled_train`
lr_model = lr.fit(scaled_train)

### **Intercept ($\theta_0$) and Coefficients ($\theta_1, \ldots, \theta_n$)**

In [0]:
print("Intercept: " + str(lr_model.intercept))
print("Coefficients: " + str(lr_model.coefficients))

#### **Measuring performance on the Training Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
training_result = lr_model.summary
print("***** Training Set *****")
print("RMSE: {:.3f}".format(training_result.rootMeanSquaredError))
print("R2: {:.3f}".format(training_result.r2))
print("Adjusted R2: {:.3f}".format(training_result.r2adj))
print("***** Training Set *****")

In [0]:
# Let's apply the same feature scaling to the test set
scaled_test = scaler_model.transform(test)

In [0]:
# Compute predictions on the test set and select only the columns of interest
lr_predictions = lr_model.transform(scaled_test)
lr_predictions.select("features", "std_features", "prediction", "label").show(5)

#### **Measuring performance on the Test Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
test_result = lr_model.evaluate(scaled_test)
print("***** Test Set *****")
print("RMSE: {:.3f}".format(test_result.rootMeanSquaredError))
print("R2: {:.3f}".format(test_result.r2))
print("Adjusted R2: {:.3f}".format(test_result.r2adj))
print("***** Test Set *****")

### **Observations**

Since we are using simple OLS linear regression (i.e., no regularization term is included), we obtain *exactly* the same result whether we do feature scaling or not. Of course, the interpretation of the coefficients are much more convenient when features are standardized.

Generally speaking, the regression coefficient $\theta_i$ indicates the effect of a change in $x_i$ on the target variable $y$ with all of the other features $x_j$ unchanged. The measurement units of regression coefficient $\theta_i$ are units of $y$ per unit of $x_i$. For example, if $y$ is the dollar amount of sales and $x_1$ is the number of people in the sales force, then $\theta_1$ is expressed as units of dollars of sales per person. Now, suppose that the next regression coefficient, $\theta_2$, is in units of dollars of sales per number of total miles traveled by the sales force. The question of which is more important to sales, staffing level or travel budget, cannot be answered by comparing $\theta_1$ to $\theta_2$ because dollars per person and dollars per mile are not directly comparable. Feature standardization solves this problem by expressing each "unit" of each feature $x_i$ as a statistical unit equal to one standard deviation.

Another reason why feature standardization may be helpful is when the optimizer we use to solve OLS is **not** analyitical (i.e., Normal Equation) but iterative (e.g., L-BFGS or Gradient Descent); in this case standardization helps the numerical method to converge faster.

Finally, the effect of feature standardization is instead appreciable whenever we aim to solve a regularized version of the regularization problem (e.g., L1- or L2-norm regularization).

## **Summary of the main findings from Data Exploration step**

From our exploratory data analysis above, we observed:
- ~Both continuous and categorical features~ (**DONE**);
- ~Different scales of feature values~ (**NOT NECESSARY UNLESS WE USE REGULARIZATION OR NEED A BETTER INTERPRETATION OF COEFFICIENTS**);
- Several outliers on many features (see the boxplots above).

### **Handling Outliers**

In this notebook, we will not deal with this specific issue. However, I strongly suggest you to try apply one of the techniques we mentioned in class:
- outliers removal (_hard_ approach);
- winsorization (_soft_ approach).

Recall that feature winsorization works as follow: you choose a certain percentile value (e.g., 90) and for each feature $f_i$ you replace any value $x_i$ which falls either below the 5-th percentile or above the 95-th percentile with the 5-th and the 95-th percentile, respectively.

# **Putting All Together**

In the following, we try to summarize the whole pipeline making use also of $K$-fold cross validation to get a better estimate of the generalization performance of our model.

In [0]:
# This function defines the general pipeline for linear regression
def linear_regression_pipeline(train, 
                               numerical_features, 
                               categorical_features, 
                               target_variable, 
                               with_std=True,
                               with_mean=True,
                               k_fold=5):

    from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    from pyspark.ml.evaluation import RegressionEvaluator
    from pyspark.ml.regression import LinearRegression
    from pyspark.ml import Pipeline

    # Configure a linear regression pipeline, which consists of the following stages: 
    # 1) convert categorical features to numerical ones
    # 2) standardize feature values (optional)
    # ... add any other custom transformation here ...
    # n) fit a linear regressor model

    # 1.a Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 1.b Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")

    # 1.c Assemble all the features into a single vector
    assembler = VectorAssembler(inputCols=encoder.getOutputCols() + numerical_features, outputCol="features")

    # 2.a Create the StandardScaler
    # scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)

    # ...

    if TARGET_VARIABLE != "label":
        train = train.withColumnRenamed(TARGET_VARIABLE, "label")

    lr = LinearRegression(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    pipeline = Pipeline(stages=indexers + [encoder] + [assembler] + [lr]) # add/remove `[scaler]` to the pipeline if needed


    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for lr.regParam ($\lambda$) and 3 values for lr.elasticNetParam ($\alpha$),
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.0, 0.05, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=RegressionEvaluator(metricName="rmse"),
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = linear_regression_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: lambda=[{:.3f}]; alpha=[{:.3f}] ".format(m.stages[-1]._java_obj.getRegParam(), m.stages[-1]._java_obj.getElasticNetParam()))
        print("***************************************\n")

In [0]:
# Call the function above|
summarize_all_models(cv_model.subModels)

In [0]:
for i, avg_rmse in enumerate(cv_model.avgMetrics):
    print("Avg. RMSE computed across k-fold cross validation for model setting #{:d}: {:3f}".format(i+1, avg_rmse))

### **Measuring performance on the Training Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
# `bestModel` is the best resulting model according to K-fold cross validation, which is also entirely retrained on the whole `train_df`
training_result = cv_model.bestModel.stages[-1].summary
print("***** Training Set *****")
print("RMSE: {:.3f}".format(training_result.rootMeanSquaredError))
print("R2: {:.3f}".format(training_result.r2))
print("Adjusted R2: {:.3f}".format(training_result.r2adj))
print("***** Training Set *****")

### **Using the best model from $K$-fold cross validation to make predictions**

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_df = test_df.withColumnRenamed("price", "label")
predictions = cv_model.transform(test_df)

In [0]:
predictions.select("features", "prediction", "label").show(5)

In [0]:
def evaluate_model(predictions, metric="rmse"):
    
    from pyspark.ml.evaluation import RegressionEvaluator

    evaluator = RegressionEvaluator(labelCol="label",
                                    predictionCol="prediction",
                                    metricName=metric)

    return evaluator.evaluate(predictions)

In [0]:
def r2_adj(predictions):
    
    r2 = evaluate_model(predictions, metric="r2")
    r2_adj_score = (1 - (1 - r2) * ((predictions.count() - 1) / (predictions.count() - predictions.select('features').first()[0].size - 1)))

    return r2_adj_score

### **Measuring performance on the Test Set: $\text{RMSE}$ and $\text{R}^2$ statistic**

In [0]:
print("***** Test Set *****")
print("RMSE: {:.3f}".format(evaluate_model(predictions)))
print("R2: {:.3f}".format(evaluate_model(predictions, metric="r2")))
print("Adjusted R2: {:.3f}".format(r2_adj(predictions)))
print("***** Test Set *****")